In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [ ]:
import pandas as pd

# Load dataset
data = pd.read_csv('gdrive/My Drive/March-26-2024-MEDIQA-CORR-Official-Test-Set.csv', encoding = "ISO-8859-1")




In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text ID    925 non-null    object
 1   Text       925 non-null    object
 2   Sentences  925 non-null    object
dtypes: object(3)
memory usage: 21.8+ KB


In [ ]:
!pip install fuzzywuzzy


In [ ]:
import pandas as pd
from difflib import SequenceMatcher
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def add_best_match_column(df, patient_id_col, text_col, threshold=0.5):
    """
    Adds a new column to the DataFrame containing the triplets of best matching case reports based on custom similarity scoring of case report texts.

    Args:
        df (pandas.DataFrame): The DataFrame to modify.
        patient_id_col (str): The name of the column containing the patient ID.
        text_col (str): The name of the column containing the case report text.
        threshold (float, optional): The minimum similarity score for considering a pair. Defaults to 0.5.

    Returns:
        pandas.DataFrame: The modified DataFrame with the new column.
    """

    df['best_match'] = 'NOMATCH'
    for i in range(len(df)):
        text1 = df.loc[i, text_col]
        for j in range(i + 1, len(df)):
            text2 = df.loc[j, text_col]
            score = custom_similarity_score(text1, text2)  # Use custom similarity scoring function
            if score >= threshold:
                if df.loc[i, 'best_match'] == 'NOMATCH':
                    df.at[i, 'best_match'] = f'({i},{j},{score})'
                else:
                    # Check if new match has a higher score
                    old_score = float(df.at[i, 'best_match'].split(',')[2].replace(')', ''))
                    if score > old_score:
                        df.at[i, 'best_match'] = f'({i},{j},{score})'
                    else:
                        continue
                if df.loc[j, 'best_match'] == 'NOMATCH':
                    df.at[j, 'best_match'] = f'({j},{i},{score})'
                else:
                    # Check if new match has a higher score
                    old_score = float(df.at[j, 'best_match'].split(',')[2].replace(')', ''))
                    if score > old_score:
                        df.at[j, 'best_match'] = f'({j},{i},{score})'
    return df

def custom_similarity_score(s1, s2):
    """
    Custom similarity scoring function. Computes the similarity score based on the number of common tokens.

    Args:
        s1 (str): First string for comparison.
        s2 (str): Second string for comparison.

    Returns:
        float: Similarity score.
    """
    tokens1 = set(word_tokenize(s1.lower()))
    tokens2 = set(word_tokenize(s2.lower()))
    common_tokens = tokens1.intersection(tokens2)
    score = len(common_tokens) / min(len(tokens1), len(tokens2))
    return score
'''
# Sample DataFrame with textual data
data = {
    'ID': ['ms-test-523', 'ms-test-524', 'ms-test-525', 'ms-test-526', 'ms-test-527'],
    'Text': [
        "A 27-year-old woman who resides in an area endemic for chloroquine-resistant P. falciparum malaria presents to the physician with fatigue, malaise, and episodes of fever with chills over the last 5 days. She mentions that she has episodes of shivering and chills on alternate days that last for approximately 2 hours, followed by high-grade fevers; then she has profuse sweating and her body temperature returns to normal. She also mentions that she is currently in her 7th week of pregnancy. The physical examination reveals the presence of mild splenomegaly. Patient is given quinine sulfate plus doxycycline after a peripheral blood smear confirms the diagnosis of P. falciparum infection.",
        "A 27-year-old woman who resides in an area endemic for chloroquine-resistant P. falciparum malaria presents to the physician with fatigue, malaise, and episodes of fever with chills over the last 5 days. She mentions that she has episodes of shivering and chills on alternate days that last for approximately 2 hours, followed by high-grade fevers; then she has profuse sweating and her body temperature returns to normal. She also mentions that she is currently in her 7th week of pregnancy. The physical examination reveals the presence of mild splenomegaly. A peripheral blood smear confirms the diagnosis of P. falciparum infection. Patient is given quinine sulfate plus clindamycin.",
        "\"Three days after admission to the intensive care unit for septic shock and bacteremia from a urinary tract infection, a 34-year-old woman has persistent hypotension. Her blood cultures are positive for Escherichia coli, for which she has been receiving appropriate antibiotics since admission. She has no history of any serious illness. She does not use illicit drugs. Current medications include norepinephrine, ceftriaxone, and acetaminophen. She appears well. Her temperature is 37.5 C (99.5 F), heart rate is 96/min, and blood pressure is 85/55 mm Hg. Examination of the back shows costovertebral tenderness bilaterally. Examination of the thyroid gland shows no abnormalities. Laboratory studies show: Hospital day 1 Hospital day 3 Leukocyte count 18,500/mm3 10,300/mm3 Hemoglobin 14.1 mg/dL 13.4 mg/dL Serum Creatinine 1.4 mg/dL 0.9 mg/dL Fasting glucose 95 mg/dL 100 mg/dL TSH 1.8 µU/mL T3, free 0.1 ng/dL T4, free 0.9 ng/dL Repeat blood cultures are negative. A chest X-ray shows no abnormalities. The toxic drugs are removed.\"",
        "A 19-year-old female college student is brought to the emergency department by her friend who found her unusually lethargic and unresponsive. Her friend informs the physician that she has asthma, a history of 'some eating disorder,' and depression for which she is taking an unknown medication. He says she drinks socially and smokes marijuana. On physical exam, the patient is somnolent and ill-appearing. The patient's temperature is 99.5 F (37.5 C), blood pressure is 85/60 mmHg, pulse is 110/min, and respirations are 9/min. Her skin is hot, dry and flushed, and pupils are dilated. Patient is given naloxone after an EKG is done and the results are shown in Figure A.",
        "A 19-year-old female college student is brought to the emergency department by her friend who found her unusually lethargic and unresponsive. Her friend informs the physician that she has asthma, a history of 'some eating disorder,' and depression for which she is taking an unknown medication. He says she drinks socially and smokes marijuana. On physical exam, the patient is somnolent and ill-appearing. The patient's temperature is 99.5 F (37.5 C), blood pressure is 85/60 mm"
  ]
}'''

# Example usage
#df = pd.DataFrame(data)
data = add_best_match_column(data, 'Text ID', 'Text')
print(data)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


         Text ID                                               Text  \
0      ms-test-0  A 29-year-old internal medicine resident prese...   
1      ms-test-1  A 29-year-old internal medicine resident prese...   
2      ms-test-2  A 7-year-old boy is brought by his parents to ...   
3      ms-test-3  A 7-year-old boy is brought by his parents to ...   
4      ms-test-4  A 42-year-old morbidly obese woman presents to...   
..           ...                                                ...   
920  uw-test-323  Ms. <NAME/> is also being managed for hypokale...   
921  uw-test-324  Ms. <NAME/> is a <AGE/> yo female with h/o nec...   
922  uw-test-325  <NAME/> is being managed for an abdominopelvic...   
923  uw-test-326  Mr. <NAME/> is a <AGE/>M with AML s/p PBSCT on...   
924  uw-test-327  Mr. <NAME/> is an <AGE/> YO man with 6 weeks b...   

                                             Sentences  \
0    0 A 29-year-old internal medicine resident pre...   
1    0 A 29-year-old internal m

In [ ]:
#data.to_csv('gdrive/My Drive/processedoutput.csv', index=False)


In [ ]:
import pandas as pd

data = pd.read_csv('gdrive/My Drive/processedoutput.csv', encoding = "ISO-8859-1")


In [ ]:
from difflib import SequenceMatcher
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
# Function to compute similarity between two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
input_string = data['Sentences'][0]
# Tokenize the input string into sentences
input_sentences = sent_tokenize(input_string)

# Single sentence to match against
target_sentence = "Culture tests indicate Neisseria gonorrhoeae."

# Initialize variables to store best match index and similarity score
best_match_index = -1
best_similarity = 0

# Iterate over each sentence in the list along with its index
for index, sentence in enumerate(input_sentences):
    # Compute similarity between current sentence and target sentence
    similarity_score = similar(sentence, target_sentence)
    # Update best match if similarity score is higher
    if similarity_score > best_similarity:
        best_similarity = similarity_score
        best_match_index = index

print("Best match index:", best_match_index)
print("Similarity:", best_similarity)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Best match index: 9
Similarity: 0.4186046511627907


In [ ]:
import joblib

# Save the trained pipeline
#joblib.dump(pipeline, 'gdrive/My Drive/TFIDFCorrectmodel.pkl')

# Load the trained pipeline
loaded_model = joblib.load('gdrive/My Drive/MediFact/MediFact1-ErrorSVMTraining.pkl')
loaded_model2 = joblib.load('gdrive/My Drive/MediFact/MediFact1-CorrectSVMTraining.pkl')


In [ ]:
def process_pair(sample1, sample2, error_detection_model, correct_sentence_detection_model, vectorizer):
  error_flag = [1, 0]  # 1 for sample 1, 0 for sample 2
  error_sentence_index = [0, -1]  # 0 for sample 1 (always the first sentence), -1 for sample 2
  correct_sentence = [None, None]  # None for not found

  # Error detection in sample 1 using first model
  sentence_features = vectorizer.transform([sample1["Sentences"][0]])
  error_prediction = error_detection_model.predict(sentence_features)[0]

  # Apply error detection model and handle potential error
  if error_prediction == 1:
    error_flag[0] = 1
    error_sentence_index[0] = 0

    # Use correct sentence detection model on sample 2 sentences
    candidate_sentences = sample2["Sentences"]  # All sentences in sample 2 are potential corrections
    corrected_sentence_candidates = []
    for sentence in candidate_sentences:
      sentence_features = vectorizer.transform([sentence])
      correction_prediction = correct_sentence_detection_model.predict(sentence_features)[0]
      if correction_prediction == 1:
        corrected_sentence_candidates.append(sentence)  # Collect potential corrections

    # Choose the best correction (optional: implement selection criteria)
    if len(corrected_sentence_candidates) > 0:
      # You can define a selection logic based on similarity, position, or other factors
      correct_sentence[0] = corrected_sentence_candidates[0]

  # No change needed for sample 2 sentences (already processed in previous pairs)
  correct_sentence[1] = None

  return error_flag, error_sentence_index, correct_sentence

In [ ]:
from difflib import SequenceMatcher
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def find_best_match(input_text, target_sentence):
    # Tokenize the input string into sentences
    input_sentences = sent_tokenize(input_text)

    # Function to compute similarity between two strings
    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()

    # Initialize variables to store best match index and similarity score
    best_match_index = -1
    best_similarity = 0

    # Iterate over each sentence in the input text along with its index
    for index, sentence in enumerate(input_sentences):
        # Compute similarity between current sentence and target sentence
        similarity_score = similar(sentence, target_sentence)
        # Update best match if similarity score is higher
        if similarity_score > best_similarity:
            best_similarity = similarity_score
            best_match_index = index
        gg= get_first_integer_word(sentence)


    return gg

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def get_first_integer_word(input_string, idih):
    # Split the string into words
    words = input_string.split()

    # Check if the first word is an integer
    if words[0].isdigit():
      a = 8
      string_part = input_string.split(' ', 1)[1][:-1]
      return words[0], string_part
    else:
      return idih, input_string

In [ ]:
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

def find_best_match(sentence, paragraph):
    # Tokenize paragraph into sentences
    sentences = nltk.sent_tokenize(paragraph)

    # Add given sentence to the list of sentences
    sentences.append(sentence)

    # Vectorize sentences
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(sentences)

    # Calculate pairwise cosine similarity
    similarity_matrix = cosine_similarity(vectors)

    # Find index of the most similar sentence
    most_similar_index = similarity_matrix[-1, :-1].argmax()

    # Return the most similar sentence
    return sentences[most_similar_index]

# Example
paragraph = "A 4-year-old boy is brought to his pediatrician by his mother for a cough. radiograph. His mother reports that he has had a loud cough and hoarse voice for the past 3 days. He also had a runny nose and mild fever last week. He was born with Tetralogy of Fallot and underwent surgery as an infant. He has been followed regularly by a pediatric cardiologist. There are no other sick contacts in the household. His temperature is 100.9 F (38.3 C), blood pressure is 105/65 mmHg, pulse is 110/min, and respirations are 24/min. Parainfluenza is determined as the causal pathogen of the patient's condition after an anteroposterior radiograph of the neck is shown in Figure A. A lung radiograph is normal. "
sentence = " A lung radiograph is normal. Respiratory syncytial virus is the causal pathogen of the patient's condition."

best_match = find_best_match(sentence, paragraph)
print("Best matching sentence:", best_match)


Best matching sentence: Parainfluenza is determined as the causal pathogen of the patient's condition after an anteroposterior radiograph of the neck is shown in Figure A.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def is_single_sentence(text):
    # Count the occurrences of '.', '?', and '!' in the text
    punctuation_count = sum(text.count(char) for char in ['.', '?', '!'])

    # If there's only one punctuation mark, it's likely a single sentence
    return punctuation_count == 1

In [ ]:
Error_Flag = []
Error_Sentence_Index = []
Corrected_Sentence = []
processed_indices = set()

for i, row in data.iterrows():
    if i in processed_indices:
        continue  # Skip processing this row if it's already processed

    paragraph = row['Text']

    if row['best_match'] != 'NOMATCH':
        best_match_index = int(row['best_match'].split(',')[1])
        best_match_paragraph = data.iloc[best_match_index]['Text']

        predicted_answer = loaded_model.predict([paragraph])[0]
        print(predicted_answer)
        if is_single_sentence(predicted_answer):
            Error_Flag.append(1)
            input_text = row['Sentences']
            lines = [line.strip() for line in input_text.split("\n")]
            lines = list(filter(None, lines))
            error_match_index = find_best_match_index(lines, predicted_answer)
            pri, sentos = get_first_integer_word(error_match_index[1], error_match_index[0])
            Error_Sentence_Index.append(pri)

            processed_indices.add(i)  # Add current row index to processed indices
            processed_indices.add(i + 1)  # Add next row index to processed indices
            print(i)
            if i + 1 < len(data):
                next_row = data.iloc[i + 1]  # Move to the next row
                speci = next_row['Text']
                best_match = find_best_match(speci, sentos)
                Corrected_Sentence.append(best_match)
                Corrected_Sentence.append('NA')
                Error_Flag.append(0)
                Error_Sentence_Index.append(-1)


            else:
                Corrected_Sentence.append(sentos)
                continue  # Skip processing the rest of the loop for this row

    else:
        Corrected_Sentence.append('NA')
        Error_Flag.append(0)
        Error_Sentence_Index.append(-1)

data['Error Flag'] = Error_Flag
data['Error Sentence'] = Error_Sentence_Index
data['Corrected Sentence'] = Corrected_Sentence


Suspected of Alzheimer’s disease.
0
CT scan of chest is ordered.
2
Patient is diagnosed with placental abruption.
4
CT scan of chest is ordered.
6
Patient is diagnosed with placental abruption.
8
Albuterol was administered.
10
Patient is diagnosed with placental abruption.
12
A CT scan of the abdomen with contrast is ordered.
15
Suspected of Alzheimer’s disease.
17
Patient is diagnosed with placental abruption.
19
A CT scan of the abdomen with contrast is ordered.
21
A CT scan of the abdomen with contrast is ordered.
23
CT scan of chest is ordered.
25
Patient is diagnosed with mitral valve regurgitation.
27
Patient is diagnosed with mitral valve regurgitation.
29
Patient is diagnosed with placental abruption.
31
A CT scan of the abdomen with contrast is ordered.
33
Suspected of Alzheimer’s disease.
35
A CT scan of the abdomen with contrast is ordered.
37
Patient is diagnosed with mitral valve regurgitation.
39
Patient is diagnosed with placental abruption.
41
CT scan of chest is ordere

KeyboardInterrupt: 

In [ ]:
data

,Text ID,Text,Sentences,best_match,Error Flag,Error Sentence,Corrected Sentence
0,ms-test-0,A 29-year-old internal medicine resident prese...,0 A 29-year-old internal medicine resident pre...,"(0,1,0.978494623655914)",1,11,The rest of the examination is normal
1,ms-test-1,A 29-year-old internal medicine resident prese...,0 A 29-year-old internal medicine resident pre...,"(1,0,0.978494623655914)",0,-1,NA
2,ms-test-2,A 7-year-old boy is brought by his parents to ...,0 A 7-year-old boy is brought by his parents t...,"(2,3,0.9662921348314607)",1,9,Staphylococcus aureus is the causative agent o...
3,ms-test-3,A 7-year-old boy is brought by his parents to ...,0 A 7-year-old boy is brought by his parents t...,"(3,2,0.9662921348314607)",0,-1,NA
4,ms-test-4,A 42-year-old morbidly obese woman presents to...,0 A 42-year-old morbidly obese woman presents ...,"(4,5,0.9672131147540983)",1,4,The patient is afebrile and is in no apparent ...
5,ms-test-5,A 42-year-old morbidly obese woman presents to...,0 A 42-year-old morbidly obese woman presents ...,"(5,4,0.9672131147540983)",0,-1,NA
6,ms-test-6,A 4-year-old boy is brought to his pediatricia...,0 A 4-year-old boy is brought to his pediatric...,"(6,7,0.9662921348314607)",1,9,Respiratory syncytial virus is the causal path...
7,ms-test-7,A 4-year-old boy is brought to his pediatricia...,0 A 4-year-old boy is brought to his pediatric...,"(7,6,0.9662921348314607)",0,-1,NA
8,ms-test-8,A 2-year-old male presents to the emergency de...,0 A 2-year-old male presents to the emergency ...,"(8,9,0.9814814814814815)",1,5,The responsible microbe is isolated and its st...
9,ms-test-9,A 2-year-old male presents to the emergency de...,0 A 2-year-old male presents to the emergency ...,"(9,8,0.9814814814814815)",0,-1,NA


In [ ]:
def modify_text(text):
    # Example modification: replace 'sentence' with 'text'
    return text.replace('\n', 'text')

# Update the 'Text' column with modified values
data['Corrected Sentence'] = data['Corrected Sentence'].apply(modify_text)

In [ ]:
# Select columns to save
selected_columns = ['Text ID', 'Error Flag', 'Error Sentence', 'Corrected Sentence']

# Save selected columns as a text file
data[selected_columns].to_csv('gdrive/My Drive/prediction.txt', index=False, sep=' ', header=False)

print("Selected columns saved to 'selected_columns.txt'")

Selected columns saved to 'selected_columns.txt'
